In [1]:
import pandas as pd
import json
import numpy as np
import datetime
import math
import openpyxl
import string
import tkinter as tk
from tkinter import filedialog
from tkinter import colorchooser
import locale


## select file

In [ ]:
root = tk.Tk()
root.withdraw()

default_directory = " D:/GitHub/planningManipulation"

file_path = filedialog.askopenfilename(
    title="Select an Excel File",
    initialdir=default_directory,
    filetypes=[("Excel files", "*.xlsm"),("Excel files", "*.xlsx"), ("Excel files", "*.xls")]
)
# Print selected file path
if file_path:
    print(f"Selected file: {file_path}")
else:
    print("No file selected.")

### fetch .xlsm and retrieve every sheets content

In [ ]:
xls = pd.ExcelFile(file_path)

sheet_names = xls.sheet_names
sheets_content = []
sheet_head = []

for sheet_name in sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
    print(df.iloc[0, 0])
    if df.iloc[0, 0] != sheet_name:
        df = df.iloc[1:]
        sheet_head.append(True)
    else :
        sheet_head.append(False)
    sheets_content.append(df)

workbook = openpyxl.load_workbook(file_path)


### Convert into dict

In [ ]:
sheet_number = 1

sheet_dict_lists = sheets_content[sheet_number].to_dict(orient='list')
print(sheet_dict_lists)

### fetch name row numbers

In [ ]:
name = 'lorine'
UC_name = name.upper()
rows_with_name = []

for index, value in enumerate(sheet_dict_lists[0]):
    if value == UC_name:
        rows_with_name.append(index)

rows_with_name


### fetch week number row index

In [ ]:
row_with_week = []
print(sheet_names[sheet_number])
for index, value in enumerate(sheet_dict_lists[0]):
    if value == sheet_names[sheet_number]:
        row_with_week.append(index)

row_with_week

### fetch dates table

In [ ]:
locale.setlocale(locale.LC_TIME, 'fr_FR')
week_dates = []

for week in row_with_week :
    week_dates.append(sheet_dict_lists[1][week].strftime("%d %B %Y"))

week_dates

### fetch hours day by day

In [ ]:
#1 to 24 column key fetch and numbers @ name rows

# np array (2 dim)

planning = np.full((6, 27), '', dtype=str)
i = 0

for name in rows_with_name :
    for half_hour in range (1, 25) :
        planning[i, half_hour-1] = sheet_dict_lists[list(sheet_dict_lists.keys())[half_hour]][name]
    i+=1

planning

### fetch name cell color

In [ ]:
sheet = workbook[sheet_names[sheet_number]]

sheet_offset = 0
if(sheet_head[sheet_number]):
    sheet_offset = 2
else:
    sheet_offset = 1

for i in range(0, len(rows_with_name)):
    cell = sheet[f"A{rows_with_name[i]+sheet_offset}"]
    if isinstance(cell.fill.start_color.rgb, str):
        bg_color = cell.fill.start_color.rgb
        if bg_color == "FFFF0000":
            planning[i, 23] = '1'
            planning[i, 24] = '1'
            planning[i, 25] = '1'

planning

        

### create start and end times

In [ ]:
def workFrame(planning) :
    frames = []
    half = np.arange(16, 44, 1)
    isBegin = False
    row = [0,0]
    for i in range(0, half.size-1) :
        if(planning[i] == '1' and not isBegin) :
            isBegin = True
            row[0] = half[i]/2
        if(planning[i] != '1' and isBegin) :
            isBegin = False
            row[1] = half[i]/2
            frames.append([row[0], row[1]])

    return frames

print(workFrame(planning[4]))

In [ ]:
data = []
date = 0
start_time = datetime.datetime(2024, 1, 1, 0, 0)
days_of_week = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi']

alphabet_enum = list(string.ascii_uppercase)

sheet_offset = 0
if(sheet_head[sheet_number]):
    sheet_offset = 2
else:
    sheet_offset = 1

for day in planning :
    frame = workFrame(day)
    if frame == []:
        start = start_time + datetime.timedelta(hours=8)
        finish = start_time + datetime.timedelta(hours=20)
        activity_name = f"{days_of_week[date]} {week_dates[date]}"
        tache = "off"
        for i in range (0, day.size-4):
            cell = sheet[f"{alphabet_enum[i+1]}{rows_with_name[date]+sheet_offset}"]
            if isinstance(cell.fill.start_color.rgb, str):
                bg_color = cell.fill.start_color.rgb
                if bg_color == "FFFFFF00":
                    tache = "vacances"
                elif bg_color == "FF92D050":
                    tache = "repos"
                elif bg_color == "FFFF99CC":
                    tache = "arrêt"
        data.append(dict(Work=activity_name, Start=start, Finish=finish, Tâche=tache))
    for interval in frame :
        start_minute = 0
        if interval[0]%1 != 0:
            start_minute = 30
        start = start_time + datetime.timedelta(hours=math.floor(interval[0]), minutes=start_minute)
        finish_minute  = 0
        if interval[1]%1 != 0:
            finish_minute = 30
        finish = start_time + datetime.timedelta(hours=math.floor(interval[1]), minutes=finish_minute)
        activity_name = f"{days_of_week[date]} {week_dates[date]}"
        data.append(dict(Work=activity_name, Start=start, Finish=finish, Tâche="travail"))
    date +=1
data

#FORMAT bg_color
    

## graphical interface

In [ ]:
import plotly.express as px
import pandas as pd


# Data with hour-based tasks (use a dummy date like '2024-01-01')
df = pd.DataFrame(data)

# Convert Start and Finish to datetime format
df['Start'] = pd.to_datetime(df['Start'])
df['Finish'] = pd.to_datetime(df['Finish'])

# Calculate the duration of each task and total time elapsed
df['Duration'] = df['Finish'] - df['Start']
task_a_df = df[df['Tâche'] == "travail"]

# Sum the total duration for "Task A"
total_duration_task_a = task_a_df['Duration'].sum()

# Retrieve total time in hours for "Task A"
total_hours_task_a = total_duration_task_a.total_seconds() / 3600

# Define the exact order of tasks as they appear in the data
task_order = df["Work"].tolist()

color_map = {
    "travail": "rgb(141,237,217)",
    "off": "rgb(253,88,110)",
    "repos": "rgb(238,9,121)",
    "vacances": "rgb(192,15,191)",
    "arrêt": "rgb(174,225,242)"
}

fig = px.timeline(
    df,
    x_start="Start",
    x_end="Finish",
    y="Work",
    color="Tâche",
    category_orders={"Work": task_order},
    color_discrete_map=color_map
)

# Adjust x-axis to show tick marks at 30-minute intervals
fig.update_layout(
    xaxis=dict(
        tickformat="%H:%M",  # Format to show only hours and minutes
        title="Heure",
        side="top",  # Move the x-axis to the top
        dtick=1800000  # 30 minutes in milliseconds (30 * 60 * 1000)
    ),
    title=f"Total heures : {total_hours_task_a} h"
)

fig.show()